In [1]:
import time
import re

#  load library (url 이미지 다운로드)
import urllib.request
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

driver = webdriver.Chrome('/usr/sbin/chromedriver')

driver.implicitly_wait(3)
driver.get('https://www.foodfly.co.kr/restaurants')

# 한식[선택]
driver.find_element_by_xpath("/html/body/header/div[3]/form/div/div[2]").click()

# 웹 스크롤 다운 구현
elem = driver.find_element_by_tag_name("body")
no_of_pagedowns = 1
while no_of_pagedowns < 40:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.6)
    no_of_pagedowns += 1

# 팝업창 나올 시 해결
try:
    alert = driver.switch_to_alert()
    alert.accept()
    pass
except:
    pass

html = driver.page_source

soup = BeautifulSoup(html, 'lxml')

# 한식 데이터(이미지) 수집
soup = BeautifulSoup(html, 'lxml')
img = soup.select('#restaurants > .restaurant-list .restaurant_box .restaurant_box_bg')
count = 1
for i in img:
    img_url = i.get_text("url", strip=True)
    img_url_parse = img_url[4:-1]
    urllib.request.urlretrieve(img_url_parse, f'{count}.korea_food_img.jpg')
    count += 1

# 한식 데이터(식당명) 수집
korea_store_list = []
korea_store_name = soup.select('#restaurants > .restaurant-list .restaurant_box .restaurant_info .restaurant_name')
for name in korea_store_name:
    store_name = f'{name.string}'
    korea_store_list.append(store_name)

for i, name in enumerate(korea_store_list):
    print(i, name)


    
html = driver.page_source # 현재 페이지 주소를 반환    
ele_id = driver.find_element_by_id("restaurants")
html_content = ele_id.find_element_by_css_selector('div.restaurant-list').get_attribute('innerHTML')

soup = BeautifulSoup(html_content, 'lxml')

store_href = []
for url in soup:
    url_page = url.select('a')
    for url_href in url_page:
        a_href = url_href['href']
        store_href.append(a_href)


# 각 가게 링크에 접속 후 파싱
# 가게주소
store_address=[]
# 가게 소개
store_info_list = []
# 가게 원산지 정보
store_origin_infos = []
# 가게 최소금액
store_moneyss = []
# 가게 음식 메뉴
store_menus = []
# 음식 가격
store_prices = []
# 음식 정보
store_descriptions = []

num = 1
for p in store_href:
    try:
        driver.get(p)
        print(f'---{num}번 식당파싱 시작 ---')
        num += 1
        html2 = driver.page_source
        soup = BeautifulSoup(html2, 'lxml')
    except:
        print(f'{num}번 오류')
        pass
    # 주소뽑기
    try:
        store_address = soup.select('#restaurant-show > .clearfix > .left-section .pull-left p')[1]
        store_address_str = store_address.get_text("서울", strip=True)
        store_address_detail = store_address_str[6:]
        store_address.append(store_address_detail)
    except:
        pass
    print(store_address_detail)
    print('주소 뽑기 성공')
    print('=========================================')
    
    # 가게소개 뽑기
    store_info = soup.select('#restaurant-show > .restaurant-info > p')
    
    description = ''
    for x in store_info:
        store_info_str = x.get_text(strip=True)
        content = store_info_str + '\n'
        description += content
#         store_info_list.append(store_info_str)
    print(description)
    print('가게 소개 뽑기 성공')
    print('=========================================')
    
    # 가게 원산지 정보 뽑기
    store_origin = soup.select_one('div.restaurant-info').find_next_sibling('div')
    origin_content = store_origin.select('p')
    for w in origin_content:
        store_origin_info = w.get_text(strip=True)
        store_origin_infos.append(store_origin_info)
        
    print(store_origin_info)
    print('가게 원산지 뽑기 성공')
    print('=========================================')    
    
    # 가게최소금액 뽑기
    store_least_money_list = soup.select('.main-info-sub > p > span.info-header + span')
    for a in store_least_money_list:
        store_least_money = store_least_money_list[3]
        store_least_money_result = store_least_money.get_text(":", strip=True)[2:8]
        store_split = re.split(',', store_least_money_result)
        store_money = ''
        for i in store_split:
            store_money += i
    store_moneys=int(store_money)
    store_moneyss.append(store_moneys)
    print(store_moneys)
    print('최소 금액 성공')
    print('=========================================')
    
    store_details_menu = soup.select('.left-section > .menu-category > .row-menu')
    for menu in store_details_menu:
        # 음식 메뉴
        store_menu = menu.select_one(".pull-left").find_next('strong').get_text(strip=True)
        store_menus.append(store_menu)
        print(store_menu)
        print('음식 메뉴 성공')
        print('=========================================')
        
        # 음식 정보
        store_description = menu.select_one('.pull-left > span').get_text("", strip=True)
        store_descriptions.append(store_description)
        print(store_description)
        print('음식 정보 성공')
        print('=========================================')
        
        # 음식 가격
        store_price = menu.select_one("span.calculate-price").get_text(strip=True)
        a = re.split(',', store_price)
        aprice = ''
        for i in a:
            aprice += i
        price = int(aprice)
        store_prices.append(price)
        print(price)
        print('음식 가격 성공')
        print('=========================================')
      
    
    print('================== 가게 하나 끝 ==================')
#     print(store_menu, price)
#     print(store_description)
    
#     print(store_address)
#     print(store_info_list)
#     print(store_moneyss)
#     print(store_menus)
#     print(store_prices)
#     print(store_descriptions)
    
    # 팝업나올시 해결
    try:
        alert = driver.switch_to_alert()
        alert.accept()
        pass
    except:
        pass
    
    driver.back()
    driver.implicitly_wait(8)
    
    
print('**************끝났따!!!!!!!!!!!!!!!!!*********************')

# 가게 식당명
print(korea_store_list)
# 가게 주소
print(store_address)
# 가게 소개
print(store_info_list)
# 가게 원산지 정보
print(store_origin_infos)
# 가게 최소금액
print(store_moneyss)
# 가게 음식 메뉴
print(store_menus)
# 음식 가격
print(store_prices)
# 음식 정보
print(store_descriptions)

/home/hanyonghee/.local/share/virtualenvs/foodfly-av6wuyWs/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: use driver.switch_to.alert instead


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?